# To-do

# Daten zur Raumgliederung
In diesem Notebook werden die Informationen zum durchschnittlichen CO2-Ausstoss der in einer Gemeinde registrierten Personenwagen mit Daten zur Raumgliederung ergänzt. Diese werden vom [Bundesamt für Statistik zur Verfügung gestellt](https://www.bfs.admin.ch/bfs/de/home/statistiken/querschnittsthemen/raeumliche-analysen/raeumliche-gliederungen/raeumliche-typologien.html).

Details zur Einteilung in städtisch, ländlich und intermediär: https://www.bfs.admin.ch/bfs/de/home/statistiken/querschnittsthemen/raeumliche-analysen/raeumliche-gliederungen/raeumliche-typologien.html

Einkommensdaten pro Steuerpflichtige: https://www.atlas.bfs.admin.ch/maps/13/de/15829_9164_8282_8281/24774.html

In [1]:
# Importieren der nötigen Erweiterungen. 
import pandas as pd
from requests import post, get

## Datenimport

### Hauptdatensatz

In [2]:
# Die Hauptdaten werden eingelesen.
df = pd.read_csv('co2_daten_gefiltert_gruppiert.csv', index_col='BFS-Gemeinde-Nr')

In [3]:
# Das offizielle Gemeindeverzeichnis von 2019 wird eingelesen. 
df_gemeinden = pd.read_excel('Rohdaten/Gemeindestand_2019.xlsx')
df_gemeinden.set_index('BFS Gde-nummer', inplace=True, drop=True)
df_gemeinden = df_gemeinden[['Gemeindename', 'Kanton']].copy()

In [4]:
# Die Gemeindenamen werden im Hauptdatensatz ergänzt. 
df = pd.merge(df_gemeinden, df, left_index=True, right_index=True, how='right')

In [5]:
#Welche Gemeinden aus dem Hauptdatensatz existierten 2019 nicht mehr? (BFS-Nummer nicht in Gemeindestand 2019,
#ergo NaN bei den Spalten Gemeindename und Kanton, die von df_gemeinden kommen)
df[(df['Gemeindename'].isna())]

,Gemeindename,Kanton,ort,CO2,anz_fahrzeuge
BFS-Gemeinde-Nr,,,,,
36,NaN,NaN,Oberstammheim,151.482143,560
42,NaN,NaN,Unterstammheim,153.072765,481
44,NaN,NaN,Waltalingen,155.461126,373
134,NaN,NaN,Hütten,159.259481,501
140,NaN,NaN,Schönenberg ZH,158.807941,1083
142,NaN,NaN,Wädenswil,152.918081,9839
664,NaN,NaN,Golaten,158.217172,198
3691,NaN,NaN,Hinterrhein,166.586207,29
3693,NaN,NaN,Nufenen,169.393939,66


Fazit: Nur eine Gemeinde aus dem CH-Media-Land betroffen, sie kann von Hand fusioniert werden (Attelwil!)

### Integration von Attelwil in Reitnau

In [6]:
df[df['Gemeindename'] == 'Reitnau']

,Gemeindename,Kanton,ort,CO2,anz_fahrzeuge
BFS-Gemeinde-Nr,,,,,
4281,Reitnau,AG,Reitnau,156.763158,722


In [7]:
anz_fahrzeuge_attelwil_reitnau = 200 + 722

#Gewichteter Mittelwert des CO2 beider Ortsteile
co2_attelwil_reitnau = 159.905000 * (200/anz_fahrzeuge_attelwil_reitnau) + 156.763158 * (722/anz_fahrzeuge_attelwil_reitnau)

In [8]:
df.loc[4281] = ['Reitnau', 'AG', 'Reitnau', co2_attelwil_reitnau, anz_fahrzeuge_attelwil_reitnau]

In [9]:
#Kontrolle, ob Reitnau richtig erfasst ist
df[df['Gemeindename'] == 'Reitnau']

,Gemeindename,Kanton,ort,CO2,anz_fahrzeuge
BFS-Gemeinde-Nr,,,,,
4281,Reitnau,AG,Reitnau,157.444686,922


### Einkommen

In [10]:
col_names = ['BFS-Nummer', 'Gemeindename', 'Steuerbares Einkommen total', 'Steuerbares Einkommen pro Kopf']
df_einkommen = pd.read_excel('Rohdaten/Einkommen_AG_2017.xlsx', names=col_names, skiprows=5)
del df_einkommen['Steuerbares Einkommen total']
df_einkommen = df_einkommen.loc[:2254].copy()
df_einkommen['BFS-Nummer'] = df_einkommen['BFS-Nummer'].astype(int)
df_einkommen['Steuerbares Einkommen pro Kopf'] = df_einkommen['Steuerbares Einkommen pro Kopf'].astype(float)
df_einkommen.set_index('BFS-Nummer', inplace=True)

In [11]:
#Merge
df = pd.merge(df, df_einkommen[['Steuerbares Einkommen pro Kopf']], left_index=True, right_index=True, how='left')

### Raumtypologien

In [12]:
# Die Daten zu den Raumtypologien des BfS werden importiert. Wichtig: Daten von 2019, weil Gemeindestand oben von 2019
url = 'https://www.agvchapp.bfs.admin.ch/de/typologies/results/xls?SnapshotDate=01.01.2019&SelectedTypologies%5B0%5D=HR_GDETYP2012&SelectedTypologies%5B1%5D=HR_MONT2019'
r = post(url)
df_raumtypologien = pd.read_excel(r.content, skiprows=1)
df_raumtypologien = df_raumtypologien.dropna(subset=['BFS Gde-nummer'])
df_raumtypologien['BFS Gde-nummer'] = df_raumtypologien['BFS Gde-nummer'].apply(lambda x: int(x))
df_raumtypologien.set_index('BFS Gde-nummer', inplace=True, drop=True)

In [13]:
# Statt der Variablen werden die Kategoriennamen verwendet. 
temp_dict = {1: 'städtisch', 2: 'städtische und ländliche Merkmale', 3: 'ländlich'}
df_raumtypologien['Städtische / Ländliche Gebiete'] = df_raumtypologien['Städtische / Ländliche Gebiete'].astype('int')
df_raumtypologien['Städtische / Ländliche Gebiete'] = df_raumtypologien['Städtische / Ländliche Gebiete'].map(temp_dict)

temp_dict = {11: 'Städtische Gemeinde einer grossen Agglomeration', 
             12: 'Städtische Gemeinde einer mittelgrossen Agglomeration', 
             13: 'Städtische Gemeinde einer kleinen oder ausserhalb einer Agglomeration', 
             21: 'Periurbane Gemeinde hoher Dichte', 
             22: 'Periurbane Gemeinde mittlerer Dichte',
             23: 'Periurbane Gemeinde geringer Dichte',
             31: 'Ländliche Zentrumsgemeinde',
             32: 'Ländliche zentral gelegene Gemeinde', 
             33: 'Ländliche periphere Gemeinde'}

df_raumtypologien['Gemeindetypologie 2012 (9 Typen)'] = df_raumtypologien['Gemeindetypologie 2012 (9 Typen)'].astype('int')
df_raumtypologien['Gemeindetypologie 2012 (9 Typen)'] = df_raumtypologien['Gemeindetypologie 2012 (9 Typen)'].map(temp_dict)

In [14]:
# Die Raumtypologien und der Hauptdatensatz werden zusammengeführt. 
df = pd.merge(df, df_raumtypologien[['Bezirks-nummer', 'Bezirksname', 'Städtische / Ländliche Gebiete', 'Gemeindetypologie 2012 (9 Typen)']], 
                             left_index=True, right_index=True, how='right')

Check: Für drei Gemeinden, die seit 2019 durch Fusionen entstanden sind, sind keine Werte aus dem Hauptdatenset vorhanden. Dies, weil in den Hauptdaten noch die alten Einzelgemeinden drin sind. Betrifft keine Gemeinde aus dem CH-Media-Land.

In [15]:
df[(df['Gemeindename'].isna()) & (df['CO2'].isna())]

,Gemeindename,Kanton,ort,CO2,anz_fahrzeuge,Steuerbares Einkommen pro Kopf,Bezirks-nummer,Bezirksname,Städtische / Ländliche Gebiete,Gemeindetypologie 2012 (9 Typen)
BFS Gde-nummer,,,,,,,,,,
292,NaN,NaN,NaN,NaN,NaN,NaN,102.0,Andelfingen,ländlich,Ländliche zentral gelegene Gemeinde
293,NaN,NaN,NaN,NaN,NaN,NaN,106.0,Horgen,städtisch,Städtische Gemeinde einer grossen Agglomeration
3714,NaN,NaN,NaN,NaN,NaN,NaN,1851.0,Viamala,ländlich,Ländliche periphere Gemeinde


### Berggebiete

In [16]:
# Die Daten zu den Bergregionen des BfS werden importiert. 
url = 'https://www.agvchapp.bfs.admin.ch/de/typologies/results/xls?SnapshotDate=01.01.2019&SelectedTypologies%5B0%5D=HR_MONT2019'
r = post(url)
df_bergregionen = pd.read_excel(r.content, skiprows=1)
df_bergregionen = df_bergregionen.dropna(subset=['BFS Gde-nummer'])
df_bergregionen['BFS Gde-nummer'] = df_bergregionen['BFS Gde-nummer'].apply(lambda x: int(x))
df_bergregionen.set_index('BFS Gde-nummer', inplace=True, drop=True)

In [17]:
# Statt der Variablen werden die Kategoriennamen verwendet. 
temp_dict = {0: 'Gemeinde ausseralb der Berggebiete', 1: 'Gemeinde im Berggebiet'}
df_bergregionen['Berggebiete'] = df_bergregionen['Berggebiete'].astype('int')
df_bergregionen['Berggebiete'] = df_bergregionen['Berggebiete'].map(temp_dict)

In [18]:
# Die Raumtypologien und der Hauptdatensatz werden zusammengeführt. 
df = pd.merge(df, df_bergregionen[['Berggebiete']], left_index=True, right_index=True, how='left')

### Städte

In [19]:
# Die Daten zu den Städten des BfS werden importiert. 
url = 'https://www.agvchapp.bfs.admin.ch/de/typologies/results/xls?SnapshotDate=01.01.2019&SelectedTypologies%5B0%5D=HR_STADTE2012'
r = post(url)
df_städte = pd.read_excel(r.content, skiprows=1)
df_städte = df_städte.dropna(subset=['BFS Gde-nummer'])
df_städte['BFS Gde-nummer'] = df_städte['BFS Gde-nummer'].apply(lambda x: int(x))
df_städte.set_index('BFS Gde-nummer', inplace=True, drop=True)

In [20]:
# Statt der Variablen werden die Kategoriennamen verwendet. 
temp_dict = {0: 'keine Stadt', 1: 'Stadt'}
df_städte['Statistische Städte 2012'] = df_städte['Statistische Städte 2012'].astype('int')
df_städte['Statistische Städte 2012'] = df_städte['Statistische Städte 2012'].map(temp_dict)

In [21]:
# Die Angaben zu den Städten und der Hauptdatensatz werden zusammengeführt. 
df = pd.merge(df, df_städte[['Statistische Städte 2012']], left_index=True, right_index=True, how='left')

### Wohnbevölkerung

In [22]:
# Daten zu den Bevölkerungszahlen des BfS (2019) werden importiert und aufbereitet. (2020: 16404491)
url = 'https://www.bfs.admin.ch/bfsstatic/dam/assets/13707251/master'
r = get(url)

df_bevölkerung = pd.read_excel(r.content, skiprows=3)
df_bevölkerung = df_bevölkerung.iloc[:,:4]
df_bevölkerung = df_bevölkerung.dropna(subset=['Unnamed: 0'])
df_bevölkerung = df_bevölkerung[df_bevölkerung['Unnamed: 0'].str.startswith('.')]
df_bevölkerung['bfsnr'] = df_bevölkerung['Unnamed: 0'].apply(lambda x: int(x.split(' ')[0].replace('......', '')))
df_bevölkerung.set_index('bfsnr', inplace=True, drop=True)
del df_bevölkerung['Unnamed: 0']
df_bevölkerung.columns = ['bev_' + x.lower() for x in df_bevölkerung.columns]

In [23]:
# Die beiden Datensätze werden zusammengeführt. 
df = pd.merge(df, df_bevölkerung, left_index=True, right_index=True, how='left')

In [24]:
# Der Datensatz wird abgespeichert.
del df['ort']
df.columns = ['gemeinde', 'kanton', 'co2', 'anz_fahrzeuge', 'steuerbares einkommen pro kopf', 'bezirksnummer', 'bezirksname', 'stadt_land', 'gemeindetyp', 'berggebiet', 'stadt', 'bev_total', 'bev_mann', 'bev_frau']
df.to_csv('co2_gemeinden_typologien.csv')

## Export aller Daten

In [25]:
df.to_csv('co2_daten_schweiz_raumgliederung.csv')

## Auswertungen Aargau

In [26]:
df_ag = df[df['kanton'] == 'AG'].copy()

In [28]:
# Wie viele Gemeinden sind verzeichnet? 
len(df_ag)

211

In [29]:
# Wie hoch ist der mittlere CO2-Ausstoss der Autos pro Stand, Land und dazwischen?
df_ag.groupby(by='stadt_land')[['co2']].median().sort_values(by='co2')

,co2
stadt_land,
städtisch,156.012239
städtische und ländliche Merkmale,156.178585
ländlich,158.781513


In [30]:
#Wie sieht es bei den Städten aus?
df_ag[df_ag['stadt'] == 'Stadt'].sort_values(by='co2', ascending=False)

,gemeinde,kanton,co2,anz_fahrzeuge,steuerbares einkommen pro kopf,bezirksnummer,bezirksname,stadt_land,gemeindetyp,berggebiet,stadt,bev_total,bev_mann,bev_frau
BFS Gde-nummer,,,,,,,,,,,,,,
4040,Spreitenbach,AG,157.873667,5438.0,61254.940801,1902.0,Baden,städtisch,Städtische Gemeinde einer grossen Agglomeration,Gemeinde ausseralb der Berggebiete,Stadt,12065.0,6067.0,5998.0
4012,Suhr,AG,156.260241,4565.0,69904.408867,1901.0,Aarau,städtisch,Städtische Gemeinde einer mittelgrossen Agglom...,Gemeinde ausseralb der Berggebiete,Stadt,10476.0,5115.0,5361.0
4045,Wettingen,AG,155.773779,9336.0,77000.823211,1902.0,Baden,städtisch,Städtische Gemeinde einer mittelgrossen Agglom...,Gemeinde ausseralb der Berggebiete,Stadt,20934.0,10322.0,10612.0
4258,Rheinfelden,AG,155.570810,6009.0,78541.619048,1909.0,Rheinfelden,städtisch,Städtische Gemeinde einer grossen Agglomeration,Gemeinde ausseralb der Berggebiete,Stadt,13503.0,6707.0,6796.0
4082,Wohlen (AG),AG,155.326831,8818.0,68021.741778,1903.0,Bremgarten,städtisch,Städtische Gemeinde einer kleinen oder ausserh...,Gemeinde ausseralb der Berggebiete,Stadt,16613.0,8419.0,8194.0
4254,Möhlin,AG,155.054221,5828.0,72811.316445,1909.0,Rheinfelden,städtische und ländliche Merkmale,Periurbane Gemeinde hoher Dichte,Gemeinde ausseralb der Berggebiete,Stadt,11047.0,5558.0,5489.0
4095,Brugg,AG,154.867355,4938.0,72886.653582,1904.0,Brugg,städtisch,Städtische Gemeinde einer mittelgrossen Agglom...,Gemeinde ausseralb der Berggebiete,Stadt,11229.0,5578.0,5651.0
4280,Oftringen,AG,153.556202,7909.0,62858.336123,1910.0,Zofingen,städtisch,Städtische Gemeinde einer mittelgrossen Agglom...,Gemeinde ausseralb der Berggebiete,Stadt,14101.0,7121.0,6980.0
4001,Aarau,AG,153.555941,9510.0,74765.250929,1901.0,Aarau,städtisch,Städtische Gemeinde einer mittelgrossen Agglom...,Gemeinde ausseralb der Berggebiete,Stadt,21773.0,10713.0,11060.0


In [31]:
# Ein Blick auf die Gemeinden mit besonders tiefem und besonders hohem CO2-Durchschnittswert. 
df_ag.sort_values(by='co2', ascending=True)

,gemeinde,kanton,co2,anz_fahrzeuge,steuerbares einkommen pro kopf,bezirksnummer,bezirksname,stadt_land,gemeindetyp,berggebiet,stadt,bev_total,bev_mann,bev_frau
BFS Gde-nummer,,,,,,,,,,,,,,
4221,Abtwil,AG,149.517241,551.0,73147.858942,1908.0,Muri,ländlich,Periurbane Gemeinde geringer Dichte,Gemeinde ausseralb der Berggebiete,keine Stadt,964.0,483.0,481.0
4237,Oberrüti,AG,149.995031,805.0,76600.465839,1908.0,Muri,ländlich,Periurbane Gemeinde geringer Dichte,Gemeinde ausseralb der Berggebiete,keine Stadt,1542.0,784.0,758.0
4255,Mumpf,AG,150.673888,877.0,69617.073171,1909.0,Rheinfelden,städtische und ländliche Merkmale,Periurbane Gemeinde mittlerer Dichte,Gemeinde ausseralb der Berggebiete,keine Stadt,1553.0,802.0,751.0
4105,Mandach,AG,151.984456,193.0,71591.780822,1904.0,Brugg,ländlich,Ländliche zentral gelegene Gemeinde,Gemeinde ausseralb der Berggebiete,keine Stadt,334.0,179.0,155.0
4230,Buttwil,AG,152.006536,765.0,75882.884615,1908.0,Muri,ländlich,Ländliche zentral gelegene Gemeinde,Gemeinde ausseralb der Berggebiete,keine Stadt,1240.0,602.0,638.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4004,Densbüren,AG,164.852097,453.0,67091.470588,1901.0,Aarau,ländlich,Periurbane Gemeinde geringer Dichte,Gemeinde ausseralb der Berggebiete,keine Stadt,721.0,360.0,361.0
4176,Schwaderloch,AG,164.984199,443.0,66371.621622,1906.0,Laufenburg,ländlich,Ländliche zentral gelegene Gemeinde,Gemeinde ausseralb der Berggebiete,keine Stadt,685.0,364.0,321.0
4322,Wislikofen,AG,165.773399,203.0,68090.000000,1911.0,Zurzach,ländlich,Periurbane Gemeinde geringer Dichte,Gemeinde ausseralb der Berggebiete,keine Stadt,353.0,174.0,179.0


In [32]:
#Wir fügen einen kleinen Text pro Gemeinde hinzu. Liegt der CO2-Wert über dem Durchschnitt der jeweiligen Typologie?
av_stadt = df_ag[df_ag['stadt_land'] == 'städtisch']['co2'].mean()
av_int = df_ag[df_ag['stadt_land'] == 'städtische und ländliche Merkmale']['co2'].mean()
av_land = df_ag[df_ag['stadt_land'] == 'ländlich']['co2'].mean()

satzanfang = 'liegt der durchschnittliche CO2-Ausstoss der Autos '

def satz_generator(elem):
    if elem['stadt_land'] == 'städtisch':
        if elem['co2'] > av_stadt:
            return  satzanfang + 'über dem Durchschnitt der städtischen Aargauer Gemeinden.'
        elif elem['co2'] == av_stadt:
            return satzanfang + 'im Durchschnitt der städtischen Aargauer Gemeinden.'
        else:
            return satzanfang + 'unter dem Durchschnitt der städtischen Aargauer Gemeinden.'
    elif elem['stadt_land'] == 'städtische und ländliche Merkmale':
        if elem['co2'] > av_int:
            return satzanfang + 'über dem Durchschnitt der städtisch-ländlich geprägten Aargauer Gemeinden.'
        elif elem['co2'] == av_int:
            return satzanfang + 'im Durchschnitt der städtisch-ländlich geprägten Aargauer Gemeinden.'
        else:
            return satzanfang + 'unter dem Durchschnitt der städtisch-ländlich geprägten Aargauer Gemeinden.'
    elif elem['stadt_land'] == 'ländlich':
        if elem['co2'] > av_land:
            return satzanfang + 'über dem Durchschnitt der Aargauer Landgemeinden.'
        elif elem['co2'] == av_land:
            return satzanfang + 'im Durchschnitt der Aargauer Landgemeinden.'
        else:
            return satzanfang + 'unter dem Durchschnitt der Aargauer Landgemeinden.'

    
df_ag['text'] = df_ag.apply(satz_generator, axis=1)

df_ag['text'] = 'In ' + df_ag['gemeinde'] + ' ' + df_ag['text']

### Rangliste hinzufügen und Abweichung vom Mittelwert

In [36]:
df_ag['co2'] = df_ag['co2'].round(1)
df_ag['rank'] = df_ag['co2'].rank(method='min')

In [85]:
df_ag['abweichung'] = df_ag['co2'] - df_ag['co2'].mean()

### Import Aargauer Gemeindewappen

In [86]:
df_wappen = pd.read_csv('../Vorlagen/bfs-nummer_gemeinde_wappen_2020.csv', index_col='BFS-Nummer')

In [87]:
df_ag = pd.merge(df_ag, df_wappen, left_index=True, right_index=True, how='left')

## Aargau-Export für Datawrapper

In [41]:
#Interaktive Karte
df_ag[['gemeinde', 'co2', 'anz_fahrzeuge', 'stadt_land', 'text', 'Wappen']].to_csv('ag_total.csv')

In [88]:
#Rangliste
df_ag[['rank', 'gemeinde', 'co2', 'abweichung', 'anz_fahrzeuge']].sort_values(by='co2', ascending=False).to_csv('ag_total_tabelle.csv')

In [ ]:
#Einkommen
df_ag[['gemeinde', 'co2', 'steuerbares einkommen pro kopf', 'stadt_land']].to_csv('ag_co2_einkommen.csv')

### Kontrolle: Autos pro Einwohner

In [53]:
df_autodichte = df_ag[['gemeinde', 'anz_fahrzeuge', 'bev_total']].copy()
df_autodichte.set_index('gemeinde', inplace=True, append=True)
df_autodichte = pd.DataFrame(df_autodichte['anz_fahrzeuge'] / df_autodichte['bev_total'])
df_autodichte.rename(columns={0: 'Autodichte'}, inplace=True)
df_autodichte.sort_values(by='Autodichte')

,,Autodichte
BFS Gde-nummer,gemeinde,
4001,Aarau,0.436779
4002,Biberstein,0.540050
4003,Buchs (AG),0.478108
4004,Densbüren,0.628294
4005,Erlinsbach (AG),0.543274
...,...,...
4318,Schneisingen,0.595559
4319,Siglistorf,0.669811
4320,Tegerfelden,0.608368


## Auswertung Solothurn

In [57]:
df_so = df[df['kanton'] == 'SO'].copy()

In [65]:
#Wir fügen einen kleinen Text pro Gemeinde hinzu. Liegt der CO2-Wert über dem Durchschnitt der jeweiligen Typologie?
av_berg = df_so[df_so['berggebiet'] == 'Gemeinde im Berggebiet']['co2'].mean()
av_nicht_berg = df_so[df_so['berggebiet'] == 'Gemeinde ausseralb der Berggebiete']['co2'].mean()

satzanfang = 'liegt der durchschnittliche CO2-Ausstoss der Autos '

def satz_generator_so(elem):
    if elem['berggebiet'] == 'Gemeinde im Berggebiet':
        if elem['co2'] > av_berg:
            return  satzanfang + 'über dem Durchschnitt der Solothurner Berggemeinden.'
        elif elem['co2'] == av_berg:
            return satzanfang + 'im Durchschnitt der Solothurner Berggemeinden.'
        else:
            return satzanfang + 'unter dem Durchschnitt der Solothurner Berggemeinden.'
    elif elem['berggebiet'] == 'Gemeinde ausseralb der Berggebiete':
        if elem['co2'] > av_nicht_berg:
            return satzanfang + 'über dem Durchschnitt der Solothurner Gemeinden ausserhalb der Berggebiete.'
        elif elem['co2'] == av_nicht_berg:
            return satzanfang + 'im Durchschnitt der Solothurner Gemeinden ausserhalb der Berggebiete.'
        else:
            return satzanfang + 'unter dem Durchschnitt der Solothurner Gemeinden ausserhalb der Berggebiete.'

    
df_so['text'] = df_so.apply(satz_generator_so, axis=1)

In [67]:
df_so['text'] = 'In ' + df_so['gemeinde'] + ' ' + df_so['text']

### Solothurner Wappen + Merge + Export

In [69]:
df_so_wappen = pd.read_csv('../Vorlagen/solothurner_gemeinden_wappen_2021.csv')

In [71]:
df_so_wappen.set_index('Codes:', inplace=True)

In [72]:
df_so_final = pd.merge(df_so, df_so_wappen, left_index=True, right_index=True, how='left')

In [83]:
df_so_final['co2'] = df_so_final['co2'].round(1)

In [84]:
df_so_final[['gemeinde', 'co2', 'anz_fahrzeuge', 'berggebiet', 'text', 'wappen']][df_so_final['berggebiet'] == 'Gemeinde im Berggebiet'].to_csv('so_berg.csv')
df_so_final[['gemeinde', 'co2', 'anz_fahrzeuge', 'berggebiet', 'text', 'wappen']][df_so_final['berggebiet'] == 'Gemeinde ausseralb der Berggebiete'].to_csv('so_nicht_berg.csv')

## Nationale Auswertung

In [54]:
# Gibts grössere Unterschiede zwischen den Kantonen? 
df_temp = df.groupby(by='kanton')[['co2']].median().sort_values(by='co2', ascending=False)
df_temp.to_csv('kantonsvergleich.csv')
df_temp

,co2
kanton,
AI,165.376824
GR,164.830835
VS,163.999279
SZ,163.828891
AR,163.308548
BL,163.072069
SH,161.345008
UR,160.077524
BS,159.757931


In [ ]:
# In welchen Bezirken sind Wagen mit besonders hohen oder besonders tiefen CO2-Werten registriert? 
df_temp = df_ag.groupby(by='bezirksnummer').agg({'bezirksname': 'first', 'kanton': 'first', 'gemeinde': 'count', 'co2': 'median'}).sort_values(by='co2')
df_temp.to_csv('bezirke.csv')
df_temp.sort_values(by='co2')